<a href="https://colab.research.google.com/github/jeanbouteiller-ds/tennis_prediction/blob/main/update_all_matchs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import subprocess
import sys

def install_package(package_name):
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])
        print(f"Successfully installed {package_name}")
    except subprocess.CalledProcessError:
        print(f"Failed to install {package_name}") # Install the PyGitHub library

install_package("requests")
install_package("PyGitHub")
install_package("pandas")
install_package("tqdm")

In [2]:
import requests
import io
import importlib.util
import sys
import nbformat
import pandas

# Define the URL of the nb1 notebook on GitHub
github_url = "https://raw.githubusercontent.com/jeanbouteiller-ds/tennis_prediction/main/functions_fixed_table.ipynb"

# Download the notebook as a raw .ipynb file
response = requests.get(github_url)

if response.status_code == 200:
  notebook_content = response.text

  # Parse the notebook content
  notebook = nbformat.reads(notebook_content, as_version=4)

  # Now you can execute the cells in the notebook
  for cell in notebook.cells:
      if cell.cell_type == 'code':
          exec(cell.source)

# Games Update

In [4]:
from tqdm import tqdm
import re

df_all_matchs=get_file_content("df_all_matchs.csv")
df_tournaments=get_file_content("df_tournaments.csv")

df_tournaments=df_tournaments[(pd.to_datetime(df_tournaments['tournament_date'])>=(datetime.now()-timedelta(days=15)))&
                              (pd.to_datetime(df_tournaments['tournament_date'])<=(datetime.now()+timedelta(days=15)))]

list_matchs=[]
pct_tournaments=1
nb_rows=int(pct_tournaments*len(df_tournaments))
for index,row in tqdm(df_tournaments[:int(nb_rows)].iterrows(), total=nb_rows):
  list_matchs+=find_matchs(row)

games_to_add=pd.DataFrame(list_matchs).dropna(subset='winner_name')

final_df=pd.concat([df_all_matchs,games_to_add]).drop_duplicates().reset_index(drop=True)

add_file_github('df_all_matchs.csv',final_df.to_csv(index=False))

100%|██████████| 7/7 [00:11<00:00,  1.57s/it]


In [17]:
matchs_report=get_file_content("update_all_matchs_report.txt")

number_new_tournaments=len(final_df['tournament_name'].unique())-len(df_all_matchs['tournament_name'].unique())
number_new_games=len(final_df)-len(df_all_matchs)
time_run=datetime.now()

string_return=str(time_run)+': '+str(number_new_tournaments)+' new tournaments and '+str(number_new_games)+' new games'
matchs_report.append(string_return)

add_file_github('update_all_matchs_report.txt',str(matchs_report).replace(',',',\n'))